In [2]:
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast


pd.set_option('display.precision',6)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [3]:
PATH='C:/Projects_0/hack/UFC/'          #Local
#PATH = ''                              #Kaggle

In [4]:
#fighters_df = pd.read_csv(PATH+'data/0.fighters_raw.csv', index_col=0)
fighters_df = pd.read_csv('data/0.fighters_raw.csv', index_col=0)
fighters_df['dateOfBirth'] = pd.to_datetime(fighters_df['dateOfBirth'])
fighters_df['birthYear'] = fighters_df['dateOfBirth'].dt.year
use_cols = ['name', 'armSpan', 'city', 'country', 'dateOfBirth',
            'height', 'legSwing', 'weight', 'timezone', 'weightCategory.name']

In [5]:
# В основном бойцы 16 ранга, но есть и другие...
fighters_df[fighters_df['rank']!=16].T

,241,281,405,441,475,484,618,628,686,1924,3414,3431,3457
armSpan,177.8,165.1,172.72,177.8,170.18,165.1,167.64,175.26,175.26,170.18,NaN,177.8,NaN
avgFightTime,13:18,09:00,12:07,07:09,11:58,12:08,12:40,00:00,15:00,NaN,NaN,NaN,NaN
city,Tijuana,Hanford,NaN,NaN,Rio de Janeiro,San Antonio,Kansas City,NaN,NaN,NaN,NaN,NaN,NaN
country,Mexico,United States,Brazil,United States,Brazil,United States,United States,Brazil,New Zealand,Russia,NaN,NaN,NaN
dateOfBirth,1993-12-07 00:00:00,1992-03-21 00:00:00,1987-12-18 00:00:00,1990-01-15 00:00:00,1990-04-16 00:00:00,1984-07-31 00:00:00,1986-12-24 00:00:00,1995-10-17 00:00:00,1993-03-26 00:00:00,1992-10-09 00:00:00,1970-01-01 00:00:00,1992-08-16 00:00:00,1993-10-27 00:00:00
disciplines,[],[],[],[],[],[],[],[],[],[],[],[],[]
draws,1,0,0,0,0,0,1,0,0,1,0,0,0
height,170.18,167.64,165.1,172.72,165.1,162.56,170.18,172.72,162.56,167.64,165.1,175.26,165.1
id,242,282,406,442,476,485,619,629,687,1932,3423,3440,3466
knockdownsPerFight,0.23,0.33,1.03,0.0,0.25,0.36,0.12,0.0,0.5,0.0,NaN,NaN,NaN


In [6]:
df = pd.read_csv('data/0.events_raw.csv', index_col=0)
df = df.set_index('id').sort_index()

df['eventDate.date'] = pd.to_datetime(df['eventDate.date'])
df = df.sort_values('eventDate.date').reset_index()
df['fighters'] = df['fighters'].apply(lambda x: ast.literal_eval(x))

In [7]:
df.shape

(7100, 20)

In [8]:
df.columns = [i.replace('.','_') for i in df.columns]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       7100 non-null   int64         
 1   avgOdds                  7100 non-null   object        
 2   city                     7061 non-null   object        
 3   completed                7100 non-null   bool          
 4   country                  7064 non-null   object        
 5   duration                 6942 non-null   float64       
 6   eventDate_date           7100 non-null   datetime64[ns]
 7   eventDate_timezone       7100 non-null   object        
 8   eventDate_timezone_type  7100 non-null   int64         
 9   fighterId_1              7100 non-null   int64         
 10  fighterId_2              7100 non-null   int64         
 11  fighters                 7100 non-null   object        
 12  link                     7100 non-

In [10]:
def parse_odds(df):  # Parse odds from avgOdds dict
    # Добавить кэффы в df
    for i in df.index[:]:
        avgodds = df.loc[i, 'avgOdds']

        if avgodds == '[]':
            continue

        # Преобразую данные о коэффах из строки
        ids1, odd1 = int(avgodds.split()[1].split(',')[0]), float(avgodds.split()[3].split('}')[0])
        ids2, odd2 = int(avgodds.split()[5].split(',')[0]), float(avgodds.split()[7].split('}')[0])

        # Проверить совпадает ли fighterId_1 с ids1 (который в avgOdds)
        if df.loc[i, 'fighterId_1'] == ids1:
            df.at[i, 'odd1'] = odd1
            df.at[i, 'odd2'] = odd2

        if df.loc[i, 'fighterId_1'] == ids2:
            df.at[i, 'odd1'] = odd2
            df.at[i, 'odd2'] = odd1

    return df.drop('avgOdds', axis=1)


df = parse_odds(df)
"""Удаление боёв без данных о шансах ("odds") считаю не нужным."""
#df = df[~df['odd1'].isna()].reset_index()

# When fighterID_1 wins, then Winner == 1
df['winner'] = df['winnerId'] == df['fighterId_1']

df.shape

(7100, 22)

**Significant Strike Accuracy** - результативность силовых ударов (не включают простые удары в клинче)
sigStrikesAccuracy = accentedHitsSuccessful / accentedHitsTotal

**StrikesDefendPercent**- результативность защиты: количество силовых ударов, не достигших цели
significantStrikesDefendPercent = 1 - accentedHitsSuccessful_opp / accentedHitsTotal_opp

**Strikes Landed per Minute** - количество ударов в минуту за карьеру.
significantStrikesLandPerMinute = sigStrikesAccuracy / duration * 60
Чтобы учесть предыдущее не известное количество боев можно обновлять этот параметр по с коэффициентом взвешенности 0,1
0,9 * предыдущее значение коэффициента + 0,1 * коэффициент за этот бой.

**Strikes Missed Per Minute**
significantStrikesMissedPerMinute = accentedHitsOppSuccessful / duration * 60
тот же самый подход

submissionAttemptsPer15minutes = ?

**takedowns Accuracy**
takedownsAccuracy = takedownSuccessful / takedownTotal

**takedowns Defend Percent**
takedownsDefendPercent = 1 - takedownSuccessfu_opp / takedownTotal_opp

**takedownsPer15min**
takedownsPer15min = takedownSuccessful / duration * 15 * 60

In [11]:
df['eventDate_date'].min(), df['eventDate_date'].max()

(Timestamp('1993-11-12 00:00:00'), Timestamp('2021-03-06 00:00:00'))

## Задача 1: Парсинг данных
* по примеру из baseline.
* проверка данных на пропуски, заполнение
* добавление данных из таблицы fighters : рост, вес, ранг, год рождения, количество побед

In [12]:
def parse_data_from_fight(fightStats, duration):
    if len(fightStats) == 0:
        return [np.nan]*26

    hitsTotal = fightStats.get('hitsTotal')
    hitsSuccessful = fightStats.get('hitsSuccessful')
    accentedHitsTotal = fightStats.get('accentedHitsTotal')
    accentedHitsSuccessful = fightStats.get('accentedHitsSuccessful')
    takedownTotal = fightStats.get('takedownTotal')
    takedownSuccessful = fightStats.get('takedownSuccessful')
    accentedHitsPositionDistanceTotal = fightStats.get('accentedHitsPositionDistanceTotal')
    accentedHitsPositionDistanceSuccessful = fightStats.get('accentedHitsPositionDistanceSuccessful')
    accentedHitsPositionClinchTotal = fightStats.get('accentedHitsPositionClinchTotal')
    accentedHitsPositionClinchSuccessful = fightStats.get('accentedHitsPositionClinchSuccessful')
    accentedHitsPositionParterTotal = fightStats.get('accentedHitsPositionParterTotal')
    accentedHitsPositionParterSuccessful = fightStats.get('accentedHitsPositionParterSuccessful')

    try: hitsSuccessful_percent = hitsSuccessful/hitsTotal
    except ZeroDivisionError: hitsSuccessful_percent = np.nan

    try: accentedHitsSuccessful_percent = accentedHitsSuccessful/hitsTotal
    except ZeroDivisionError: accentedHitsSuccessful_percent = np.nan

    try: accentedHits_percent = accentedHitsTotal/hitsTotal
    except ZeroDivisionError: accentedHits_percent = np.nan

    try: takedownSuccessful_percent = takedownSuccessful/takedownTotal
    except ZeroDivisionError: takedownSuccessful_percent = np.nan

    try: accentedHitsPositionDistanceSuccessful_percent = accentedHitsPositionDistanceTotal/accentedHitsPositionDistanceSuccessful
    except ZeroDivisionError: accentedHitsPositionDistanceSuccessful_percent = np.nan

    try: accentedHitsPositionClinchSuccessful_percent = accentedHitsPositionClinchTotal/accentedHitsPositionClinchSuccessful
    except ZeroDivisionError: accentedHitsPositionClinchSuccessful_percent = np.nan

    try: accentedHitsPositionParterSuccessful_percent = accentedHitsPositionParterTotal/accentedHitsPositionParterSuccessful
    except ZeroDivisionError: accentedHitsPositionParterSuccessful_percent = np.nan

    try: takedowns_to_hits = takedownSuccessful / hitsSuccessful
    except ZeroDivisionError: takedowns_to_hits = np.nan

    try: HitsPositionDistance_to_hits = accentedHitsPositionDistanceSuccessful / hitsSuccessful
    except ZeroDivisionError: HitsPositionDistance_to_hits = np.nan

    try: HitsPositionClinch_to_hits = accentedHitsPositionClinchSuccessful / hitsSuccessful
    except ZeroDivisionError: HitsPositionClinch_to_hits = np.nan

    try: HitsPositionParter_to_hits = accentedHitsPositionParterSuccessful / hitsSuccessful
    except ZeroDivisionError: HitsPositionParter_to_hits = np.nan

    hitsPM = (60 * hitsTotal) / duration
    accentedHitsPM = (60 * accentedHitsTotal) / duration
    takedownsPM = (60 * takedownTotal) / duration
    accentedHitsDistancePM = (60 * accentedHitsPositionDistanceTotal) / duration
    accentedHitsClinchPM = (60 * accentedHitsPositionClinchTotal) / duration
    accentedHitsParterPM = (60 * accentedHitsPositionParterTotal) / duration

    hitsSuccessfulPM = (60 * hitsSuccessful) / duration

    #accentedHitsSuccessfulPM = (60 * accentedHitsSuccessful) / duration  # exchange to 'significantStrikesLandPerMinute'
    takedownsSuccessfulPM = (60 * takedownSuccessful) / duration
    accentedHitsDistanceSuccessfulPM = (60 * accentedHitsPositionDistanceSuccessful) / duration
    accentedHitsClinchSuccessfulPM = (60 * accentedHitsPositionClinchSuccessful) / duration
    accentedHitsParterSuccessfulPM = (60 * accentedHitsPositionParterSuccessful) / duration

    """
    sigStrikesAccuracy = accentedHitsSuccessful / accentedHitsTotal
    significantStrikesDefendPercent = 1 - accentedHitsSuccessful_opp / accentedHitsTotal_opp
        ### this is opposite 'sigStrikesAccuracy' feature of opponent
    significantStrikesLandPerMinute = accentedHitsSuccessful / duration * 60
    significantStrikesMissedPerMinute = accentedHitsOppSuccessful / duration * 60
    takedownsAccuracy = takedownSuccessful / takedownTotal
    takedownsDefendPercent = 1 - takedownSuccessful_opp / takedownTotal_opp
        ### this is opposite 'takedownsAccuracy' feature of opponent
    takedownsPer15min = takedownSuccessful / duration * 15 * 60
    """
    try: sigStrikesAccuracy = accentedHitsSuccessful / accentedHitsTotal
    except ZeroDivisionError: sigStrikesAccuracy = np.nan

    significantStrikesLandPerMinute = (60 * accentedHitsSuccessful) / duration

    try: takedownsAccuracy = takedownSuccessful / takedownTotal
    except ZeroDivisionError: takedownsAccuracy = np.nan

    takedownsPer15min = takedownSuccessful / duration * 15 * 60

    return  hitsPM, accentedHitsPM, takedownsPM, accentedHitsDistancePM, accentedHitsClinchPM, accentedHitsParterPM, hitsSuccessfulPM, takedownsSuccessfulPM, accentedHitsDistanceSuccessfulPM, accentedHitsClinchSuccessfulPM, accentedHitsParterSuccessfulPM, hitsSuccessful_percent, accentedHitsSuccessful_percent, accentedHits_percent, takedownSuccessful_percent, accentedHitsPositionDistanceSuccessful_percent,accentedHitsPositionClinchSuccessful_percent, accentedHitsPositionParterSuccessful_percent,takedowns_to_hits, HitsPositionDistance_to_hits, HitsPositionClinch_to_hits, HitsPositionParter_to_hits, sigStrikesAccuracy, significantStrikesLandPerMinute, takedownsAccuracy, takedownsPer15min


In [13]:
fightStats_cols = ['hitsPM', 'accentedHitsPM', 'takedownsPM',
                   'accentedHitsDistancePM', 'accentedHitsClinchPM', 'accentedHitsParterPM',

                   'hitsSuccessfulPM',  'takedownsSuccessfulPM', #'accentedHitsSuccessfulPM',
                   'accentedHitsDistanceSuccessfulPM', 'accentedHitsClinchSuccessfulPM', 'accentedHitsParterSuccessfulPM',

                   'hitsSuccessful_percent', 'accentedHitsSuccessful_percent', 'accentedHits_percent',
                   'takedownSuccessful_percent', 'accentedHitsPositionDistanceSuccessful_prcent',
                   'accentedHitsPositionClinchSuccessful_percent',  'accentedHitsPositionParterSuccessful_percent',
                   'takedowns_to_hits', 'HitsPositionDistance_to_hits', 'HitsPositionClinch_to_hits',
                   'HitsPositionParter_to_hits',
                   'sigStrikesAccuracy', 'significantStrikesLandPerMinute', 'takedownsAccuracy', 'takedownsPer15min'
                   ]

fighter1_cols = [i+'_fighter1' for i in fightStats_cols]
fighter2_cols = [i+'_fighter2' for i in fightStats_cols]


In [14]:
df_stats = pd.DataFrame(index = df.index, columns=fighter1_cols + fighter2_cols) # + ['roundsCount'])

suppl_cols =  ['id', 'city', 'completed', 'country', 'duration','eventDate_date','fighterId_1', 'fighterId_2', 'name', 'rounds', 'timezone', 'weightCategory_name', 'winMethods', 'winnerId', 'odd1', 'odd2', 'winner',]

df_stats[suppl_cols] = df[suppl_cols]

In [15]:
for i in tqdm(df.index[:]):
    fighters = df.loc[i, 'fighters']
    fighterId_1, fighterId_2, duration = df.loc[i, ['fighterId_1', 'fighterId_2', 'duration']]

    # Check index of fighters in dict
    if fighters[0]['fighterId'] == fighterId_1:
        fighterId_1_ind = 0
        fighterId_2_ind = 1
    else:
        fighterId_1_ind = 1
        fighterId_2_ind = 0

    fighter1_stats = parse_data_from_fight(fighters[fighterId_1_ind]['fightStats'], duration)
    fighter2_stats = parse_data_from_fight(fighters[fighterId_2_ind]['fightStats'], duration)

    df_stats.loc[i, fighter1_cols] = fighter1_stats
    df_stats.loc[i, fighter2_cols] = fighter2_stats

df_stats.sample(5)

100%|██████████| 7100/7100 [04:42<00:00, 25.14it/s]


,hitsPM_fighter1,accentedHitsPM_fighter1,takedownsPM_fighter1,accentedHitsDistancePM_fighter1,accentedHitsClinchPM_fighter1,accentedHitsParterPM_fighter1,hitsSuccessfulPM_fighter1,takedownsSuccessfulPM_fighter1,accentedHitsDistanceSuccessfulPM_fighter1,accentedHitsClinchSuccessfulPM_fighter1,accentedHitsParterSuccessfulPM_fighter1,hitsSuccessful_percent_fighter1,accentedHitsSuccessful_percent_fighter1,accentedHits_percent_fighter1,takedownSuccessful_percent_fighter1,accentedHitsPositionDistanceSuccessful_prcent_fighter1,accentedHitsPositionClinchSuccessful_percent_fighter1,accentedHitsPositionParterSuccessful_percent_fighter1,takedowns_to_hits_fighter1,HitsPositionDistance_to_hits_fighter1,HitsPositionClinch_to_hits_fighter1,HitsPositionParter_to_hits_fighter1,sigStrikesAccuracy_fighter1,significantStrikesLandPerMinute_fighter1,takedownsAccuracy_fighter1,takedownsPer15min_fighter1,hitsPM_fighter2,accentedHitsPM_fighter2,takedownsPM_fighter2,accentedHitsDistancePM_fighter2,accentedHitsClinchPM_fighter2,accentedHitsParterPM_fighter2,hitsSuccessfulPM_fighter2,takedownsSuccessfulPM_fighter2,accentedHitsDistanceSuccessfulPM_fighter2,accentedHitsClinchSuccessfulPM_fighter2,accentedHitsParterSuccessfulPM_fighter2,hitsSuccessful_percent_fighter2,accentedHitsSuccessful_percent_fighter2,accentedHits_percent_fighter2,takedownSuccessful_percent_fighter2,accentedHitsPositionDistanceSuccessful_prcent_fighter2,accentedHitsPositionClinchSuccessful_percent_fighter2,accentedHitsPositionParterSuccessful_percent_fighter2,takedowns_to_hits_fighter2,HitsPositionDistance_to_hits_fighter2,HitsPositionClinch_to_hits_fighter2,HitsPositionParter_to_hits_fighter2,sigStrikesAccuracy_fighter2,significantStrikesLandPerMinute_fighter2,takedownsAccuracy_fighter2,takedownsPer15min_fighter2,id,city,completed,country,duration,eventDate_date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory_name,winMethods,winnerId,odd1,odd2,winner
3029,38.823529,38.823529,0.0,17.647059,0.0,21.176471,31.764706,0.0,10.588235,0.0,21.176471,0.818182,0.818182,1.0,NaN,1.666667,NaN,1.0,0.0,0.333333,0.0,0.666667,0.818182,31.764706,NaN,0.0,10.588235,10.588235,0.0,10.588235,0.0,0.0,7.058824,0.0,7.058824,0.0,0.0,0.666667,0.666667,1.0,NaN,1.5,NaN,NaN,0.0,1.0,0.0,0.0,0.666667,7.058824,NaN,0.0,3199,Denver,True,USA,17.0,2012-08-11,902,1300,UFC 150,1.0,America/Denver,Легчайший вес,['KO'],902.0,NaN,NaN,True
2572,31.8,27.0,0.6,21.2,5.8,0.0,17.2,0.0,8.0,5.4,0.0,0.540881,0.421384,0.849057,0.0,2.65,1.074074,NaN,0.0,0.465116,0.313953,0.0,0.496296,13.4,0.0,0.0,26.0,23.0,1.0,18.0,4.0,1.0,15.0,0.4,8.0,3.6,1.0,0.576923,0.484615,0.884615,0.4,2.25,1.111111,1.0,0.026667,0.533333,0.24,0.066667,0.547826,12.6,0.4,6.0,6154,Kent,True,USA,300.0,2011-06-24,1206,3018,Strikeforce Challengers 16,3.0,America/New_York,Легкий вес,['DEC'],3018.0,NaN,NaN,False
5916,72.272727,72.272727,0.0,72.272727,0.0,0.0,32.727273,0.0,32.727273,0.0,0.0,0.45283,0.45283,1.0,NaN,2.208333,NaN,NaN,0.0,1.0,0.0,0.0,0.45283,32.727273,NaN,0.0,68.181818,68.181818,0.0,58.636364,9.545455,0.0,35.454545,0.0,25.909091,9.545455,0.0,0.52,0.52,1.0,NaN,2.263158,1.0,NaN,0.0,0.730769,0.269231,0.0,0.52,35.454545,NaN,0.0,296,Fortaleza,True,Brazil,44.0,2019-02-02,234,571,UFC Fight Night 144,2.0,America/Fortaleza,Полулегкий вес,['KO'],571.0,1.74,2.29,False
6049,7.368421,6.947368,0.210526,1.684211,2.736842,2.526316,6.526316,0.0,1.052632,2.526316,2.526316,0.885714,0.828571,0.942857,0.0,1.6,1.083333,1.0,0.0,0.16129,0.387097,0.387097,0.878788,6.105263,0.0,0.0,21.263158,12.210526,0.0,5.473684,6.736842,0.0,16.421053,0.0,2.526316,5.473684,0.0,0.772277,0.376238,0.574257,NaN,2.166667,1.230769,NaN,0.0,0.153846,0.333333,0.0,0.655172,8.0,NaN,0.0,185,Saint Petersburg,True,Russia,285.0,2019-04-20,269,515,UFC Fight Night 149,1.0,Europe/Moscow,Тяжелый вес,['KO'],269.0,1.42,3.11,True
5740,22.471042,18.301158,0.926641,4.401544,1.389961,12.509653,17.142857,0.0,1.621622,1.158301,10.42471,0.762887,0.587629,0.814433,0.0,2.714286,1.2,1.2,0.0,0.094595,0.067568,0.608108,0.721

Пропуски в статистических данных думаю, заполнять не стоит. Нужно их правильно обрабатывать при аггрегировании в сумму далее
Пропуски в общей статистике - нужно попытаться исправить.

In [16]:
# drop fights, where win method is not recognised ('[]')
df_stats = df_stats[~(df_stats.winMethods == '[]')]

In [17]:
# просто удалим строки с пропущенной длительностью и количеством раундов: что с ними делать - совсем не понятно
df_stats = df_stats[~pd.isna(df_stats.duration)]

In [18]:
df_stats.isna().sum()

hitsPM_fighter1                      24
accentedHitsPM_fighter1              24
takedownsPM_fighter1                 24
accentedHitsDistancePM_fighter1      24
accentedHitsClinchPM_fighter1        24
                                   ... 
winMethods                            0
winnerId                              0
odd1                               3299
odd2                               3299
winner                                0
Length: 69, dtype: int64

In [19]:
# fill NaN in vector 'country'  as 'USA" - the most frequency value
df_stats.country.fillna('USA', inplace=True)

In [20]:
# fill NaN in vector 'city' as 'Las Vegas' and 'Rio de Janeiro' - the most frequency values per country
df_stats.loc[(df_stats.city.isnull()) & (df_stats.country=='USA'), 'city'] = 'Las Vegas'
df_stats.loc[(df_stats.city.isnull()) & (df_stats.country=='Brazil'), 'city'] = 'Rio de Janeiro'

In [21]:
df_stats.isna().sum()

hitsPM_fighter1                      24
accentedHitsPM_fighter1              24
takedownsPM_fighter1                 24
accentedHitsDistancePM_fighter1      24
accentedHitsClinchPM_fighter1        24
                                   ... 
winMethods                            0
winnerId                              0
odd1                               3299
odd2                               3299
winner                                0
Length: 69, dtype: int64

In [ ]:
df_stats.drop_duplicates(inplace=True) # удалим дубликат id=1621

### Данные о бойцах

In [22]:
list_stat = ['id', 'rank', 'height', 'weight', 'birthYear', 'wins']
df_fighters_copy = fighters_df[list_stat].copy()

In [23]:
df_fighters_copy.isna().sum(), df_fighters_copy.shape

#tem = pd.merge(df_stats, df_fighters_copy, left_on='fighterId_1', right_on='id', how='left')
#tem.isna().sum()

(id             0
 rank           0
 height       334
 weight       139
 birthYear     70
 wins           0
 dtype: int64,
 (3669, 6))

In [24]:
# fill NaN with average values
df_fighters_copy.height.fillna(df_fighters_copy.height.mean(), inplace=True)
df_fighters_copy.weight.fillna(df_fighters_copy.weight.mean(), inplace=True)
df_fighters_copy.birthYear.fillna(int(df_fighters_copy.birthYear.mean()), inplace=True)

In [25]:
df_fighters_copy.isna().sum()

id           0
rank         0
height       0
weight       0
birthYear    0
wins         0
dtype: int64

# Resume
В итоге ролучилось две таблицы df_stats и df_fighters_copy.
я полумал, что слияние таблиц будет лучше сделать после разделения датасета по бойцам, чтобы не плодить векторы признаков, которые там не нужны...

In [152]:
df_fighters_copy

,id,rank,height,weight,birthYear,wins
0,1,16,187.96,115.67,1991.0,17
1,2,16,190.50,112.04,1989.0,6
2,3,16,190.50,106.59,1981.0,20
3,4,16,182.88,92.99,1992.0,15
4,5,16,177.80,77.11,1975.0,14
...,...,...,...,...,...,...
3664,3673,16,182.88,83.91,1991.0,18
3665,3674,16,182.88,77.11,1987.0,15
3666,3675,16,170.18,65.77,1994.0,11
3667,3676,16,175.26,65.77,1990.0,13


In [28]:
df_stats.winner.value_counts()
# боец в красном углу побеждал в два раза чаще бойца в синем.
# по статистике, боец в красном углу - это фаворит боя - побеждает в 90% случае. Судьи ставят на 13% больше очков красному углу.

True     4544
False    2342
Name: winner, dtype: int64

## Задача 2: Функция расчета статистики
* кумулятивные формулы
* простая статистика
* Подключение данных в те бои, где у бойцов нет предыдущих боев, из таблицы fighters

Первый подпункт. Обработка типов побед

In [30]:
df_stats['winMethods'] = df_stats['winMethods'].apply(lambda x: x[2:-2])
y = pd.get_dummies(df_stats.winMethods, prefix='winMethods')
df_stats[y.columns] = y

In [31]:
df_stats.sample(10).T

,6364,260,6179,4000,4419,1485,4839,2173,505,169
hitsPM_fighter1,19.66805,24.736842,6.8,6.05042,34.8,67.272727,37.526502,35.2,5.813149,53.023256
accentedHitsPM_fighter1,19.66805,10.0,4.6,6.05042,29.0,44.242424,26.289753,32.2,4.775087,19.534884
takedownsPM_fighter1,0.0,4.210526,1.8,0.0,2.8,0.0,0.0,0.0,0.0,7.674419
accentedHitsDistancePM_fighter1,19.419087,8.947368,2.0,6.05042,24.4,29.090909,18.657244,26.2,2.076125,4.883721
accentedHitsClinchPM_fighter1,0.248963,0.526316,0.8,0.0,3.6,13.333333,3.816254,4.8,0.0,0.697674
...,...,...,...,...,...,...,...,...,...,...
winMethods_DQ,0,0,0,0,0,0,0,0,0,0
winMethods_KO,0,1,0,1,0,0,0,0,1,1
winMethods_NC,0,0,0,0,0,0,0,0,0,0
winMethods_SUB,1,0,0,0,0,1,1,0,0,0


In [154]:
def get_fighter_statistics_from_past(df_stats, fighterId, eventDate, item):
    '''
    Aggregate statistics from fighter from the past.
    Check both positions 1 and 2.
    '''
    temp_df = df_stats[df_stats.eventDate_date < eventDate].copy()     # датасет до даты боя
    item_df = df_stats[df_stats.id == item]                            # запись исследуемого боя

    # если датасет по итогу фильтров пустой (боец в впервый раз в базе) -
    # тогда данные оставляются из оригинальных данных боя
    first_pos_df  = temp_df[temp_df['fighterId_1'] == fighterId]
    if first_pos_df.empty:
        first_pos_df = pd.DataFrame(item_df[fighter1_cols].values)
    else:
        first_pos_df  = pd.DataFrame(first_pos_df[fighter1_cols].values)

    second_pos_df = df_stats[df_stats['fighterId_2'] == fighterId]
    if second_pos_df.empty:
        second_pos_df = pd.DataFrame(item_df[fighter2_cols].values)
    else:
        second_pos_df = pd.DataFrame(second_pos_df[fighter2_cols].values)

    joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)

    return joined_df.mean(skipna=True)

In [155]:
train_date = '2019-01-01 00:00:00'
valid_date = '2020-01-01 00:00:00'
test_date = '2021-01-01 00:00:00'

In [156]:
dates = [valid_date, test_date]
train_df_stats = df_stats[df_stats['eventDate_date'] < valid_date]
valid_df_stats = df_stats[(df_stats['eventDate_date'] >= valid_date) & (df_stats['eventDate_date'] < test_date)]
test_df_stats = df_stats[df_stats['eventDate_date'] >= test_date]

train_df_stats.shape, valid_df_stats.shape, test_df_stats.shape

((6383, 75), (422, 75), (80, 75))

In [157]:
train_df = train_df_stats.copy()
list_to_calc = [
    'fighter1_fightsAmount', 'fighter2_fightsAmount', 'fighter1_winDEC', 'fighter2_winDEC',
    'fighter1_winDQ', 'fighter2_winDQ', 'fighter1_winKO', 'fighter1_winKO',
    'fighter1_winNC', 'fighter2_winNC', 'fighter1_winSUB', 'fighter1_winSUB'
]
train_df[list_to_calc] = 0

In [158]:
def cumulative_sum(df, fighterId, vector=False):
    """
    функция расчета суммы боев и выигрышей конкретным методом
    :param df: датасет
    :param fighterId: код бойца
    :param vector: вектор признака, если не указан - считаем количество боев
    :return: сумма по выбранному вектору или сумма боев
    """

    if not vector:
        result = (df['fighterId_1'] == fighterId | (df['fighterId_2'] == fighterId)).sum(skipna=True)

    else:
        result = (((df['fighterId_1'] == fighterId) & (df[vector] == 1) & (df['winner'] == True)) |
                  ((df['fighterId_2'] == fighterId) & (df[vector] == 1) & (df['winner'] == False))).sum(skipna=True)

    if not result:
        result = 0

    return result

In [159]:
#train_df.iloc[10]
#fighterId_1, fighterId_2, max_date = train_df.loc[0, ['fighterId_1', 'fighterId_2', 'eventDate_date']]
#print(fighterId_1, fighterId_2, max_date)

#temp_df = df_stats[df_stats.eventDate_date < max_date].copy()
#temp_df
#fighter1_history_stats = get_figher_statistics_from_past(temp_df, fighterId_1)
#train_df.loc[0, fighter1_cols] = fighter1_history_stats.values

#temp_df

In [161]:
for i in tqdm(train_df_stats.index):

    fighterId_1, fighterId_2, max_date, item = train_df.loc[i, ['fighterId_1', 'fighterId_2', 'eventDate_date', 'id']]

    if max_date == df_stats.eventDate_date.min():
        continue

    temp_df = df_stats[df_stats.eventDate_date < max_date].copy()

    # строчка для обновления даты перед каждым файтом
    fighter1_history_stats = get_fighter_statistics_from_past(train_df, fighterId_1, max_date, item)
    train_df.loc[i, fighter1_cols] = fighter1_history_stats.values

    fighter2_history_stats = get_fighter_statistics_from_past(train_df, fighterId_2, max_date, item)
    train_df.loc[i, fighter2_cols] = fighter2_history_stats.values

    fighter1_fightsAmount = cumulative_sum(temp_df, fighterId_1)
    fighter2_fightsAmount = cumulative_sum(temp_df, fighterId_2)

    # подсчет типов побед Dec по каждому бойцу
    fighter1_winDEC = cumulative_sum(temp_df, fighterId_1, 'winMethods_DEC')
    fighter2_winDEC = cumulative_sum(temp_df, fighterId_2, 'winMethods_DEC')

    fighter1_winDQ = cumulative_sum(temp_df, fighterId_1, 'winMethods_DQ')
    fighter2_winDQ = cumulative_sum(temp_df, fighterId_2, 'winMethods_DQ')

    fighter1_winKO = cumulative_sum(temp_df, fighterId_1, 'winMethods_KO')
    fighter2_winKO = cumulative_sum(temp_df, fighterId_2, 'winMethods_KO')

    fighter1_winNC = cumulative_sum(temp_df, fighterId_1, 'winMethods_NC')
    fighter2_winNC = cumulative_sum(temp_df, fighterId_2, 'winMethods_NC')

    fighter1_winSUB = cumulative_sum(temp_df, fighterId_1, 'winMethods_SUB')
    fighter2_winSUB = cumulative_sum(temp_df, fighterId_2, 'winMethods_SUB')

    # внесение данных в дата сет
    train_df.loc[i, ['fighter1_fightsAmount', 'fighter2_fightsAmount']] = fighter1_fightsAmount, fighter2_fightsAmount
    train_df.loc[i, ['fighter1_winDEC', 'fighter2_winDEC']] = fighter1_winDEC, fighter2_winDEC
    train_df.loc[i, ['fighter1_winDQ', 'fighter2_winDQ']] = fighter1_winDQ, fighter2_winDQ
#    train_df.loc[i, ['fighter1_winKO', 'fighter2_winKO']] = fighter1_winKO, fighter2_winKO
    train_df.loc[i, ['fighter1_winNC', 'fighter2_winNC']] = fighter1_winNC, fighter2_winNC
#    train_df.loc[i, ['fighter1_winSUB', 'fighter2_winSUB']] = fighter1_winSUB, fighter2_winSUB

100%|██████████| 6383/6383 [19:35<00:00,  5.43it/s]


In [164]:
valid_df = valid_df_stats.copy()
list_to_calc = [
    'fighter1_fightsAmount', 'fighter2_fightsAmount', 'fighter1_winDEC', 'fighter2_winDEC',
    'fighter1_winDQ', 'fighter2_winDQ', 'fighter1_winKO', 'fighter1_winKO',
    'fighter1_winNC', 'fighter2_winNC', 'fighter1_winSUB', 'fighter1_winSUB'
]
valid_df[list_to_calc] = 0

for i in tqdm(valid_df_stats.index):

    fighterId_1, fighterId_2, max_date, item = valid_df.loc[i, ['fighterId_1', 'fighterId_2', 'eventDate_date', 'id']]

    if max_date == df_stats.eventDate_date.min():
        continue

    temp_df = df_stats[df_stats.eventDate_date < max_date].copy()

    # строчка для обновления даты перед каждым файтом
    fighter1_history_stats = get_fighter_statistics_from_past(df_stats, fighterId_1, max_date, item)
    train_df.loc[i, fighter1_cols] = fighter1_history_stats.values

    fighter2_history_stats = get_fighter_statistics_from_past(df_stats, fighterId_2, max_date, item)
    train_df.loc[i, fighter2_cols] = fighter2_history_stats.values

    fighter1_fightsAmount = cumulative_sum(temp_df, fighterId_1)
    fighter2_fightsAmount = cumulative_sum(temp_df, fighterId_2)

    # подсчет типов побед Dec по каждому бойцу
    fighter1_winDEC = cumulative_sum(temp_df, fighterId_1, 'winMethods_DEC')
    fighter2_winDEC = cumulative_sum(temp_df, fighterId_2, 'winMethods_DEC')

    fighter1_winDQ = cumulative_sum(temp_df, fighterId_1, 'winMethods_DQ')
    fighter2_winDQ = cumulative_sum(temp_df, fighterId_2, 'winMethods_DQ')

    fighter1_winKO = cumulative_sum(temp_df, fighterId_1, 'winMethods_KO')
    fighter2_winKO = cumulative_sum(temp_df, fighterId_2, 'winMethods_KO')

    fighter1_winNC = cumulative_sum(temp_df, fighterId_1, 'winMethods_NC')
    fighter2_winNC = cumulative_sum(temp_df, fighterId_2, 'winMethods_NC')

    fighter1_winSUB = cumulative_sum(temp_df, fighterId_1, 'winMethods_SUB')
    fighter2_winSUB = cumulative_sum(temp_df, fighterId_2, 'winMethods_SUB')

    # внесение данных в дата сет
    valid_df.loc[i, ['fighter1_fightsAmount', 'fighter2_fightsAmount']] = fighter1_fightsAmount, fighter2_fightsAmount
    valid_df.loc[i, ['fighter1_winDEC', 'fighter2_winDEC']] = fighter1_winDEC, fighter2_winDEC
    valid_df.loc[i, ['fighter1_winDQ', 'fighter2_winDQ']] = fighter1_winDQ, fighter2_winDQ
#    valid_df.loc[i, ['fighter1_winKO', 'fighter2_winKO']] = fighter1_winKO, fighter2_winKO
    valid_df.loc[i, ['fighter1_winNC', 'fighter2_winNC']] = fighter1_winNC, fighter2_winNC
#    valid_df.loc[i, ['fighter1_winSUB', 'fighter2_winSUB']] = fighter1_winSUB, fighter2_winSUB

100%|██████████| 422/422 [01:21<00:00,  5.20it/s]


In [168]:
valid_df

,hitsPM_fighter1,accentedHitsPM_fighter1,takedownsPM_fighter1,accentedHitsDistancePM_fighter1,accentedHitsClinchPM_fighter1,accentedHitsParterPM_fighter1,hitsSuccessfulPM_fighter1,takedownsSuccessfulPM_fighter1,accentedHitsDistanceSuccessfulPM_fighter1,accentedHitsClinchSuccessfulPM_fighter1,accentedHitsParterSuccessfulPM_fighter1,hitsSuccessful_percent_fighter1,accentedHitsSuccessful_percent_fighter1,accentedHits_percent_fighter1,takedownSuccessful_percent_fighter1,accentedHitsPositionDistanceSuccessful_prcent_fighter1,accentedHitsPositionClinchSuccessful_percent_fighter1,accentedHitsPositionParterSuccessful_percent_fighter1,takedowns_to_hits_fighter1,HitsPositionDistance_to_hits_fighter1,HitsPositionClinch_to_hits_fighter1,HitsPositionParter_to_hits_fighter1,sigStrikesAccuracy_fighter1,significantStrikesLandPerMinute_fighter1,takedownsAccuracy_fighter1,takedownsPer15min_fighter1,hitsPM_fighter2,accentedHitsPM_fighter2,takedownsPM_fighter2,accentedHitsDistancePM_fighter2,accentedHitsClinchPM_fighter2,accentedHitsParterPM_fighter2,hitsSuccessfulPM_fighter2,takedownsSuccessfulPM_fighter2,accentedHitsDistanceSuccessfulPM_fighter2,accentedHitsClinchSuccessfulPM_fighter2,accentedHitsParterSuccessfulPM_fighter2,hitsSuccessful_percent_fighter2,accentedHitsSuccessful_percent_fighter2,accentedHits_percent_fighter2,takedownSuccessful_percent_fighter2,accentedHitsPositionDistanceSuccessful_prcent_fighter2,accentedHitsPositionClinchSuccessful_percent_fighter2,accentedHitsPositionParterSuccessful_percent_fighter2,takedowns_to_hits_fighter2,HitsPositionDistance_to_hits_fighter2,HitsPositionClinch_to_hits_fighter2,HitsPositionParter_to_hits_fighter2,sigStrikesAccuracy_fighter2,significantStrikesLandPerMinute_fighter2,takedownsAccuracy_fighter2,takedownsPer15min_fighter2,id,city,completed,country,duration,eventDate_date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory_name,winMethods,winnerId,odd1,odd2,winner,winMethods_DEC,winMethods_DQ,winMethods_KO,winMethods_NC,winMethods_SUB,winMethods_,fighter1_fightsAmount,fighter2_fightsAmount,fighter1_winDEC,fighter2_winDEC,fighter1_winDQ,fighter2_winDQ,fighter1_winKO,fighter1_winKO,fighter1_winNC,fighter2_winNC,fighter1_winSUB,fighter1_winSUB
6420,46.0,46.0,0.0,45.0,1.0,0.0,15.0,0.0,14.2,0.8,0.0,0.326087,0.326087,1.0,NaN,3.169014,1.25,NaN,0.0,0.946667,0.053333,0.0,0.326087,15.0,NaN,0.0,21.2,20.0,0.0,19.0,1.0,0.0,9.2,0.0,7.6,0.6,0.0,0.433962,0.386792,0.943396,NaN,2.5,1.666667,NaN,0.0,0.826087,0.065217,0.0,0.41,8.2,NaN,0.0,39823,Las Vegas,True,USA,300.0,2020-01-18,208,272,UFC 246,3.0,America/Los_Angeles,Не определена,,208.0,1.93,1.89,True,1,0,0,0,0,1,0,0,1,4,0,0,0,0,0,0,0,0
6421,15.2,7.8,1.6,6.8,1.0,0.0,10.2,0.6,3.6,0.4,0.0,0.671053,0.263158,0.513158,0.375,1.888889,2.5,NaN,0.058824,0.352941,0.039216,0.0,0.512821,4.0,0.375,9.0,18.0,14.0,0.6,10.0,1.2,2.8,11.4,0.0,4.8,1.0,1.6,0.633333,0.411111,0.777778,0.0,2.083333,1.2,1.75,0.0,0.421053,0.087719,0.140351,0.528571,7.4,0.0,0.0,39825,Las Vegas,True,USA,300.0,2020-01-18,619,1932,UFC 246,3.0,America/Los_Angeles,Не определена,,1932.0,2.19,1.70,False,1,0,0,0,0,1,0,0,3,1,0,0,0,0,0,0,0,0
6422,12.071006,4.260355,0.710059,1.775148,0.0,2.485207,9.940828,0.35503,1.065089,0.0,1.420118,0.823529,0.205882,0.352941,0.5,1.666667,NaN,1.75,0.035714,0.107143,0.0,0.142857,0.583333,2.485207,0.5,5.325444,8.87574,4.970414,0.0,2.840237,0.35503,1.775148,6.745562,0.0,1.065089,0.35503,1.775148,0.76,0.36,0.56,NaN,2.666667,1.0,1.0,0.0,0.157895,0.052632,0.263158,0.642857,3.195266,NaN,0.0,39826,Las Vegas,True,USA,169.0,2020-01-18,550,2905,UFC 246,1.0,America/Los_Angeles,Не определена,,550.0,2.13,1.73,True,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
6423,20.2,17.6,0.0,16.2,1.4,0.0,8.8,0.0,5.2,1.0,0.0,0.435644,0.306931,0.871287,NaN,3.115385,1.4,NaN,0.0,0.590909,0.113636,0.0,0.352273,6.2,NaN,0.0,23.2,22.8,0.2,18.8,4.0,0.0,11.6,0.0,8.4,2.8,0.0,0.5,0.482759,0.982759,0.0,2.238095,1.428571,NaN,0.0,0.724138,0.241379,0.0,0.491228,11.2,0.0,0.0,39827,Las Vegas,True,USA,300.0,2020-01-18,544,2121

### Combine

In [170]:
def combine_df(to_combine_df):
    df_combined = pd.DataFrame(to_combine_df[fighter1_cols].values - to_combine_df[fighter2_cols].values,
                               index=to_combine_df.index)

    df_combined.columns = fightStats_cols
    df_combined['eventDate_date'] = to_combine_df['eventDate_date']
    df_combined['winner'] = to_combine_df['winner']
    df_combined['odd_diff'] = to_combine_df['odd1'] - to_combine_df['odd2']
    df_combined['fighter1_fightsAmount'] = to_combine_df['fighter1_fightsAmount']
    df_combined['fighter2_fightsAmount'] = to_combine_df['fighter2_fightsAmount']

    df_combined['odd1'] = to_combine_df['odd1']
    df_combined['odd2'] = to_combine_df['odd2']
    return df_combined

In [171]:
train_df_combined = combine_df(train_df)
valid_df_combined = combine_df(valid_df)

In [172]:
train_df_combined.to_csv('data/train_combined.csv')
valid_df_combined.to_csv('data/valid_combined.csv')

## Задача 3: Разделение датасета
условия:
* есть список с общими фичами (данные про бой: дата, место и т.п.)
* есть список с фичами для бойца (без учета индекса "f1_" "f2_"
* есть датасет и запись с боем для расчета.
нужно
* получить две записи с бойцами по этому бою

Далее - разделить датасет по дате до 2020 года (train) и после 2020 (valid)
(еще вариант - оставить на тест 2021 год отдельно)


## Этап II. Проверка - подбор моделей
* feature_importance
* cross correlation


## Этап III. Prediction
* расчет результата предсказания для каждого бойца
* интрепретация с учетом боя (найдем вероятности из предсказания и возьмем победителя с большим значением)
* расчет метрики по 2020 году.